<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# **Introduction to Model APIs in Keras**


Estimated time needed: **45** minutes


In this lab, we will use a simple example to introduce you to different ways of creating models in Keras. 


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li><a href="#Types of Model APIs in Keras">Types of Model APIs in Keras</a>
        <ol>
            <li><a href="#Task Definition">Task Definition</a></li>
            <li><a href="#The Sequential Model API">The Sequential Model API</a></li>
            <li><a href="#The Functional Model API">The Functional Model API</a></li>
            <li><a href="#Model Subclassing">Model Subclassing</a></li>
        </ol>
    </li>


## Objectives

After completing this lab you will be able to:

- __Understand__ different use cases for the Sequential and Functional APIs
- __Build__ custom models using sub-classing in Keras


## Setup


For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.
*   [`keras`](https://keras.io/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for loading datasets.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (like Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!pip install mlxtend` in the following code cell.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
%%capture

!pip install mlxtend
!pip install --upgrade tensorflow

### Importing Required Libraries


In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import train_test_split
seed = 7
np.random.seed(seed)


2023-05-16 07:15:05.754029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 07:15:05.940615: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-16 07:15:05.940695: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-16 07:15:07.118959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0


## Types of Model APIs in Keras 


There are three main ways of creating models in Keras.


* **The Sequential Model**: This is a straightforward way of stacking layers but is limited to having a single-input and single-output stack of layers. 
* **The Functional API**: This can support arbitrary model architectures, and is more flexible in enabling complex implementations.

* **Model subclassing**: This is a way of implementing models from scratch. This is mainly used for out-of-the-box research use cases.


In the rest of the lab, we will go through all of these ways of creating models, and walk through a use case for implementing and training each of these model architectures in Keras and Tensorflow. 


### __Task Definition__


In this lab, we will be performing a simple classification task using the 'Sonar' dataset from [UCI](http://archive.ics.uci.edu/ml/datasets/connectionist+bench+\(sonar,+mines+vs.+rocks\)?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01).


> The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.



Let's start by reading in the dataset and defining our feature and target variables.


In [3]:
dataframe = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/data/sonar.csv", header=None)

dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# encode labels
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

Now, we will split the dataset into a training and testing set with a 80:20 ratio.


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.20, random_state=42)

### **The Sequential Model API** 


The Sequential model API groups a linear stack of layers into a `tf.keras.Model`. It doesn't allow you to create models that share layers, have branches, multiple inputs, or multiple outputs. It does provide training and inference features on this model.  

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/sequential_model.png" alt="The Sequential model API" width="50%">



As you'll see, using the Sequential API is simple and straightforward, but is only appropriate for simpler, more straightforward tasks. Later in this notebook you'll spend some time building with a more flexible, powerful alternative: the Functional API.


### Create the Sequential Model


We start off by initializing the model as an instance of the Sequential class, like so: `model = Sequential()`. We follow this by adding layers, normalization layers, methods, softmax classifiers, and so on to the class, one at a time, using `model.add()`. This can be used to add different types of layer instances on top of the layer stack. Similarly, we can use `model.pop()` to remove the last layer of the model. 

Typically, the first layer receives an `input_shape` argument. This is because, in Keras, the shape of the weights is based on the shape of the inputs. The weights are only created when the model first sees some input data. After that, we can omit this argument and automate the inference of the input shape.

Note that if your model is non-linear or contains layers with multiple inputs or outputs, a Sequential model wouldn't be the right choice. 


In [5]:
def baseline_model():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [6]:
estimator = baseline_model()
estimator.summary()

2023-05-16 07:19:12.102478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-16 07:19:12.102574: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-16 07:19:12.102624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-ahmadking31j): /proc/driver/nvidia/version does not exist
2023-05-16 07:19:12.103215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                3660      
                                                                 
 dense_1 (Dense)             (None, 60)                3660      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


### Train and Evaluate the Model


After creating the model, compiling it with your choice of optimizer and loss function, and doing a sanity check on its contents, you are now ready to build.

Simply call the `.fit()` to train the model. 


In [7]:
# Write your solution here
estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 [==============================] - 1s 8ms/step - loss: 0.7003 - accuracy: 0.4759
Epoch 2/10
11/11 [==============================] - 0s 8ms/step - loss: 0.6803 - accuracy: 0.5482
Epoch 3/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6649 - accuracy: 0.6687
Epoch 4/10
11/11 [==============================] - 0s 7ms/step - loss: 0.6487 - accuracy: 0.7470
Epoch 5/10
11/11 [==============================] - 0s 8ms/step - loss: 0.6291 - accuracy: 0.7229
Epoch 6/10
11/11 [==============================] - 0s 7ms/step - loss: 0.6136 - accuracy: 0.6747
Epoch 7/10
11/11 [==============================] - 0s 10ms/step - loss: 0.5899 - accuracy: 0.7470
Epoch 8/10
11/11 [==============================] - 0s 6ms/step - loss: 0.5727 - accuracy: 0.7952
Epoch 9/10
11/11 [==============================] - 0s 5ms/step - loss: 0.5493 - accuracy: 0.7831
Epoch 10/10
11/11 [==============================] - 0s 4ms/step - loss: 0.5188 - accuracy: 0.7831


<details>
    <summary>Click here for Solution</summary>

```python
estimator.fit(X_train, y_train, epochs=10, batch_size=16)
```

</details>


After that completes, just use `.predict()` to evaluate against your test set. In this case, the `accuracy` will be used as the metric.


In [8]:
y_pred = estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 [==============================] - 0s 12ms/step


0.8571428571428571

### **The Functional Model API**


The Sequential class in Keras is typically used for implementing basic neural network architectures. The `Functional` class is typically preferred by most deep learning practitioners as it is more flexible and enables more complex implementations. It allows you to have non-linear topologies, different layers, multiple inputs and outputs. You can easily define branches in your architectures and easily share layers inside architectures. A Functional model is a graph where the nodes of the layers can connect in many more ways than one.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/functional_model.png" alt="The Sequential model API" width="50%">


All models that are implemented in `Sequential()` can easily be implemented in `Functional()`. Some examples of models that are implemented in `Functional()` include ResNet, GoogleNet, Xception. 


Both `Sequential()` and `Functional()` use the same training, evaluation, and inference functions. 

The Model class offers a built in training loop through the `.fit()` method, and a built-in evaluation loop using the `.evaluate()` method. 

The following is a simple example of fitting and evaluating a model on MNIST test data:    


We will start by loading the dataset directly using Keras.


In [9]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60, activation='relu')(inputs)
    layer2 = Dense(60, activation='relu')(layer1)
    outputs = Dense(1, activation='sigmoid')(layer2)

    model = keras.Model(inputs, outputs)
    
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [10]:
functional_estimator=functional_model()
estimator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                3660      
                                                                 
 dense_1 (Dense)             (None, 60)                3660      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


In [11]:
functional_estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 [==============================] - 1s 6ms/step - loss: 0.7086 - accuracy: 0.4157
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6842 - accuracy: 0.5181
Epoch 3/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6584 - accuracy: 0.6928
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6432 - accuracy: 0.6867
Epoch 5/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6269 - accuracy: 0.6566
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 0.5963 - accuracy: 0.7590
Epoch 7/10
11/11 [==============================] - 0s 6ms/step - loss: 0.5751 - accuracy: 0.7711
Epoch 8/10
11/11 [==============================] - 0s 6ms/step - loss: 0.5548 - accuracy: 0.7289
Epoch 9/10
11/11 [==============================] - 0s 6ms/step - loss: 0.5345 - accuracy: 0.7590
Epoch 10/10
11/11 [==============================] - 0s 5ms/step - loss: 0.5090 - accuracy: 0.7831


Similar to what we did before, evaluate the estimator's performance on the test dataset, and print out the accuracy.


In [12]:
# Write your solution here
y_pred = functional_estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 [==============================] - 0s 4ms/step


0.8809523809523809

<details>
    <summary>Click here for Solution</summary>

```python
y_pred = functional_estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)
```

</details>


### **Model Subclassing**


Next, we will learn how to make new models and classes using model sub-classing. This method is more flexible and can be used to implement out-of-box models, but this comes at a cost, it is a lot harder to utilize than `Sequential()` and `Functional()` API. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/subclassing.png" alt="The Sequential model API" width="30%">


We will implement a custom model that performs a type of convolution or pooling. In the following example, we will implement a Sequential model but define it with multiple inputs and outputs. 


In [13]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(60, activation = 'relu')
        self.dense2 = Dense(60, activation = 'relu')
        self.dense3 = Dense(1, activation = 'sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
def subclass_model():
    inputs = keras.Input(shape=(60,))
    mymodel = MyModel()
    outputs = mymodel.call(inputs)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

In [35]:
subclass_estimator = subclass_model()
subclass_estimator.fit(X_train, y_train, epochs=15, batch_size=16)

Epoch 1/15
11/11 [==============================] - 0s 4ms/step - loss: 0.6960 - accuracy: 0.5120
Epoch 2/15
11/11 [==============================] - 0s 4ms/step - loss: 0.6757 - accuracy: 0.5482
Epoch 3/15
11/11 [==============================] - 0s 4ms/step - loss: 0.6547 - accuracy: 0.6265
Epoch 4/15
11/11 [==============================] - 0s 4ms/step - loss: 0.6361 - accuracy: 0.6807
Epoch 5/15
11/11 [==============================] - 0s 4ms/step - loss: 0.6213 - accuracy: 0.6687
Epoch 6/15
11/11 [==============================] - 0s 4ms/step - loss: 0.6051 - accuracy: 0.6747
Epoch 7/15
11/11 [==============================] - 0s 4ms/step - loss: 0.5843 - accuracy: 0.7410
Epoch 8/15
11/11 [==============================] - 0s 4ms/step - loss: 0.5580 - accuracy: 0.7289
Epoch 9/15
11/11 [==============================] - 0s 4ms/step - loss: 0.5451 - accuracy: 0.7831
Epoch 10/15
11/11 [==============================] - 0s 3ms/step - loss: 0.5213 - accuracy: 0.7349
Epoch 11/15
11/11 [

In [38]:
y_pred = subclass_estimator.predict(X)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, encoded_y)

7/7 [==============================] - 0s 1ms/step


0.8413461538461539

## References


* Sequential Model: https://www.tensorflow.org/guide/keras/sequential_model

* Functional Model: https://www.tensorflow.org/guide/keras/functional


## Authors


[Kopal Garg](https://www.linkedin.com/in/gargkopal/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01)

Kopal Garg is a Masters student in computer science at the University of Toronto, and a recent Biomedical Engineer from the University of Waterloo. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2022-08-17|0.1|Kopal Garg|Created Lab|
|2022-09-08|0.1|Steve Hord|QA pass edits|


Copyright © 2022 IBM Corporation. All rights reserved.
